In [3]:
import skl2onnx
from xgboost import XGBClassifier
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn, update_registered_converter
from skl2onnx.common.shape_calculator import calculate_linear_classifier_output_shapes  # noqa
import onnxmltools
from onnxmltools.convert.xgboost.operator_converters.XGBoost import convert_xgboost  # noqa
import onnxmltools.convert.common.data_types
from skl2onnx.common.data_types import FloatTensorType, StringTensorType,Int64TensorType,BooleanTensorType
# Register xgboostclassifier converter
update_registered_converter(
    XGBClassifier, 'XGBoostXGBClassifier',
    calculate_linear_classifier_output_shapes, convert_xgboost,
    options={'nocl': [True, False], 'zipmap': [True, False]})


In [4]:
from sklearn.externals import joblib
model = joblib.load('model.joblib')

In [5]:
# Read UCI's census data sets 'adult.data' & 'adult.test'
import pandas as pd
features=['age','workclass','fnlwgt','education','edu_num','marital_status',
          'occupation','relationship','race','gender','capital_gain','capital_loss','hours_per_week',
          'native_country','label']
categorical_features= ['workclass','marital_status',
          'occupation','relationship','race','gender','native_country']
num_features=['age','fnlwgt','edu_num','capital_gain','capital_loss','hours_per_week']

df = pd.read_csv('adult.data',header=None,names=features) 
y_train = df['label']
X_train = df.drop(['label','education'],axis=1)


In [6]:
def convert_dataframe_schema(df, drop=None):
    inputs = []
    for k, v in zip(df.columns, df.dtypes):
        if drop is not None and k in drop:
            continue
        if v == 'int64' or v == 'bool':
            t = Int64TensorType([None, 1])
        elif v == 'float32':
            t = FloatTensorType([None, 1])
        else:
            t = StringTensorType([None, 1])
        inputs.append((k, t))
    return inputs
'''
to_drop = {'parch', 'sibsp', 'cabin', 'ticket',
           'name', 'body', 'home.dest', 'boat'}
'''
inputs = convert_dataframe_schema(X_train)#, to_drop)


In [8]:
# Convert SKLEARN model to ONNX format.

model_onnx = convert_sklearn(
    model, 'pipeline_xgboost',
    initial_types=inputs,
    target_opset=12)

# And save.
with open("pipeline_xgboost.onnx", "wb") as f:
    f.write(model_onnx.SerializeToString())